# core

> Main(/all) routines for `ipyfernel`

In [ ]:
#| default_exp core

In [ ]:
#| exec_doc: false

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
#| hide
from jupyter_client.manager import KernelManager
from jupyter_client.kernelspec import KernelSpecManager
import subprocess
from IPython.display import display, Image
import base64
from pathlib import Path
from IPython.core.magic import register_line_magic, register_line_cell_magic

## Remote Kernel Backend

In [ ]:
#| export
def set_ssh_config(
    port:int,                       # Port number on proxy server (e.g. bore.pub)
    user:str="",                    # Username on remote system.
    alias="remote_server_sshpyk",   # Default alias for `sshpyk`, leave it alone
    proxyname="bore.pub",           # Have tested this with bore
    config_path="~/.ssh/config",    # Shouldn't need to change this.
    ):
    config_path = Path(config_path).expanduser()
    if not config_path.exists(): config_path.touch()
    text = config_path.read_text()
    if f"Host {alias}" not in text: 
        assert user != "", "Must specify username when creating ~/.ssh/config info"
        block = f"""
Host {alias}
    HostName {proxyname}
    Port {port}
    User {user}
    BatchMode yes
    ControlMaster auto
    ControlPath ~/.ssh/sshpyk_%r@%h_%p
    ControlPersist 10m
    StrictHostKeyChecking no
    UserKnownHostsFile /dev/null
"""
        config_path.write_text(text + block)
    else:
        lines = text.splitlines()
        in_target_block = False
        for i, line in enumerate(lines):
            if line.startswith("Host "):
                in_target_block = (line == f"Host {alias}")
            elif in_target_block and line.strip().startswith("Port "):
                lines[i] = f"    Port {port}"
            elif proxyname and in_target_block and line.strip().startswith("HostName "):
                lines[i] = f"    HostName {proxyname}"
            elif user and in_target_block and line.strip().startswith("User "):
                lines[i] = f"    User {user}"
        config_path.write_text("\n".join(lines) + "\n")
    print(f'{config_path} file updated.') 

In [ ]:
# Demo that:
port = 40308
set_ssh_config(port) 

/app/data/.ssh/config file updated.


CI is failing on One of the last couple definitions, and I'm not sure why. Can you help?  The previous demo cell shouldn't be running at all because it's not being exported and I've got eval set to false. It should only be executing Yeah, the nbdev test shouldn't be running that little demo code, right?

I've even got `#| exec_doc: false` at the top of this notebook. 

So, why is it running my little demo code block? That's only for the documentation, and that should be frozen.  I do not want to have to put no test into every single cell. That's just tedious
```
+ nbdev_test --flags ''
NameError in /home/runner/work/ipyfernel/ipyfernel/nbs/00_core.ipynb:
===========================================================================

While Executing Cell #7:
Traceback (most recent call last):
  File "/opt/hostedtoolcache/Python/3.10.19/x64/lib/python3.10/site-packages/nbdev/test.py", line 52, in test_nb
    k.run_all(nb, exc_stop=True, preproc=_no_eval)
  File "/opt/hostedtoolcache/Python/3.10.19/x64/lib/python3.10/site-packages/execnb/shell.py", line 253, in run_all
    if self.exc and exc_stop: raise self.exc from None
  File "/opt/hostedtoolcache/Python/3.10.19/x64/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3579, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-1-7fa6a9c655a4>", line 2, in <module>
    set_ssh_config(port)
NameError: name 'set_ssh_config' is not defined



##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

The issue is that `#| notest` only applies to the cell it's in. You've got `#| notest` on the `set_ssh_config` **definition** cell, but not on the **demo** cell that calls it.

So what happens during `nbdev_test`:
1. The function definition is skipped (has `#| notest`)
2. The demo cell **runs** (no `#| notest`)
3. 💥 `set_ssh_config` doesn't exist because step 1 was skipped

The `eval: false` frontmatter and `#| exec_doc: false` are for **quarto/docs**, not for `nbdev_test`.

**Quick fix:** Add `#| notest` to the demo cell too.

**But** if you want to avoid sprinkling `#| notest` everywhere, you could instead **remove** `#| notest` from the function definition. Then both cells run during testing, which should work fine (assuming the SSH config write is harmless in CI).

The pattern I usually see: only use `#| notest` when a cell genuinely can't run in CI (needs hardware, credentials, etc.). If the function itself is fine to define, let it be defined so demo cells can use it.

But the demo cell isn't even being exported.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Ah, I see the confusion! There are three separate things happening here:

1. **Export** (`#| export`) → what goes into your `.py` module
2. **Docs** (`eval: false`, `#| exec_doc: false`) → what runs when building quarto docs
3. **Test** (`#| notest`) → what runs during `nbdev_test`

These are independent. A cell can be:
- Not exported, but still tested
- Exported, but not tested
- Any combination!

By default, `nbdev_test` runs **all** code cells unless they have `#| notest`. It doesn't care whether they're exported or not — it's testing your *notebook*, not your *module*.

So your demo cell runs during testing because it has no `#| notest`, even though it's not exported and won't appear in docs.

This isn't going to work. There must be some setting somewhere where we can tell it to skip testing. There's no way I can be expected to go into all 50 cells of this notebook and insert a little no-test thing onto every single one of them. What's the global way to disable the testing?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

You're right, there is a way! You can add this directive at the top of your notebook (in a code cell):

```python
#| default_exp core
#| notest
```

Wait, that won't work globally either — directives are per-cell.

Let me check the actual solution for you.Found it! [*](https://github.com/fastai/nbdev/blob/master/nbdev/test.py "nbdev/nbdev/test.py at master · AnswerDotAI/nbdev") There's a frontmatter option `skip_exec` — looking at the nbdev source code, `if str2bool(fm.get('skip_exec', False))` returns True, it skips the entire notebook during testing.

So in your raw frontmatter cell at the top, change it to:

```yaml
---
skip_exec: true
---
```

That should skip the entire notebook during `nbdev_test`. The `eval: false` you had is for quarto docs, not for testing.

In [ ]:
#| export
#| notest
def register_remote_kernel(
    kernel_name="ipyf_remote_kernel",  # Any old name will do. This is fine.
    display_name="Remote Python",      # This is just what you'll see when you look at a list.
    remote_python="/path/to/python",  # Full path of Python executable to run on remote system.
    ssh_host_alias="remote_server_sshpyk", # Same alias as was used in writing to ssh config file.
    remote_kernel_name="python3",      # Typical for Jupiter.
    language="python",                 # Probably want to leave this unless you want to try R.
    verbose=True                       # Print extra info.
    ):
    ksm = KernelSpecManager()
    registered_names = list(ksm.get_all_specs().keys())
    if kernel_name in registered_names: 
        if verbose: print(f"{kernel_name} is already a registered kernel") 
    else: 
        if verbose: print(f"{kernel_name} is not a registered kernel. We need to add it") 
        subprocess.run(["sshpyk", "add", "--kernel-name", kernel_name,
            "--display-name", display_name, "--remote-python", remote_python, "--ssh-host-alias", ssh_host_alias,
            "--remote-kernel-name", remote_kernel_name, "--language", language
        ])
        if verbose: print("Success.")


In [ ]:
register_remote_kernel(remote_python='/Users/shawley/exercises/solveit/.venv/bin/python')

ipyf_remote_kernel is already a registered kernel


In [ ]:
#| export
#| notest
_ipf_km, _ipf_kc = None, None            # "ipf" = "ipyfernel" ;-) 
def ipf_startup(kernel_name="ipyf_remote_kernel"):  
    "Start up the remote kernel"
    global _ipf_km, _ipf_kc 
    if _ipf_km is None and _ipf_kc is None: #only do this at startup
        _ipf_km = KernelManager(kernel_name=kernel_name)
        _ipf_km.start_kernel()
        _ipf_kc = _ipf_km.client()
        _ipf_kc.start_channels()
        _ipf_kc.wait_for_ready(timeout=30)
        print("Success: remote kernel started")
    else: 
        print("ipf_startup: already running")

In [ ]:
ipf_startup()

Success: remote kernel started


In [ ]:
#| export
#| notest
def _output_hook(
    msg,   #  Message obtained from remote execution
    ):
    "How to handle output from the remote kernel."
    mt = msg["msg_type"]
    content = msg.get("content", {})
    if mt == "stream":
        print(content["text"], end="", flush=True)
    elif mt == "error":
        print('\n'.join(content.get("traceback", [])))
    elif mt in ("display_data", "update_display_data"):
        data = content.get("data", {})
        if "image/png" in data:
            display(Image(base64.b64decode(data["image/png"])))
        elif "text/plain" in data:
            print(data["text/plain"])

In [ ]:
#| export
#| notest
def ipf_exec(
    code:str,           # Code to be executed
    verbose=False,      # Return details about remote execution.
    ):
    "Execute code on the remote kernel." 
    assert _ipf_kc is not None, "ipf_exec: need to run ipf_startup() first"
    result = _ipf_kc.execute_interactive(code=code, output_hook=_output_hook)
    _ipf_kc.last_result = result  # stash it for optional inspection later
    if verbose: return result

In [ ]:
code = """
import platform 
print(platform.system())
"""
ipf_exec(code)

Darwin


In [ ]:
#| export
#| notest
def ipf_shutdown(verbose=True):
    "Terminates the remote kernel"
    global _ipf_km, _ipf_kc
    if verbose: print("Shutting down remote kernel") # Note: Could make say if remote kernel is not even running.
    try:
        if _ipf_kc is not None: _ipf_kc.stop_channels()
        if _ipf_km is not None: _ipf_km.shutdown_kernel(now=True)  # 'now=True' forces immediate shutdown
    except: pass  # Don't hang on errors
    _ipf_km, _ipf_kc = None, None

In [ ]:
ipf_shutdown()

Shutting down remote kernel


# iPython Magics

In [ ]:
#| export
#| notest
def _execute_remotely(lines:list[str]):
    "Take commands from magics and send to ipf_exec"
    code = ''.join(lines)
    if 'get_ipython()' in code: return lines  # let solveit internals pass through
    # Make sure our magics execute locally
    if code.strip().startswith(('%local', '%%local', '%unset_remote', '%resume_remote', '%set_remote', '%set_sticky','%unset_sticky')):
        return lines
    return [f"ipf_exec({repr(code)})\n"]

In [ ]:
#| export
#| notest
@register_line_magic
def set_remote(line:str):
    """Setup connection to remote server, start remote server, and enable 'sticky' remote execution of code cells (even without magics).
    usage: %set_remote <port> [user]"""
    parts = line.split()
    port = int(parts[0]) if parts else 65445
    user = parts[1] if len(parts) > 1 else ""
    set_ssh_config(port, user=user) 
    try: 
        ipf_startup()
    except Exception as e: 
        print(f"Error starting up remote kernel: {e}") 
        return 

In [ ]:
%set_remote {port}

/app/data/.ssh/config file updated.


Success: remote kernel started


In [ ]:
#| export
#| notest
@register_line_cell_magic
def remote(line, cell=None):
    "remote exeuction: works as %remote and as %%remote" 
    ipf_exec(cell if cell else line)

In [ ]:
%%remote 
import socket 
hostname = socket.gethostname()   # let's make sure we're running remotely
print("Hello from",hostname) 

Hello from Chonk


In [ ]:
#| export
#| notest
@register_line_cell_magic
def local(line, cell=None):
    "local execution: works as %local and as %%local"
    get_ipython().run_cell(cell if cell else line) 

In [ ]:
%%local 
import socket 
hostname = socket.gethostname()   # let's make sure we're running remotely
print("Hello from",hostname) 

Hello from 909e93a3a71e


In [ ]:
#| export
#| notest
@register_line_magic
def unset_remote(_):
    "shutdown remote server"
    unset_sticky('')  # get rid of any input transformers (see below) 
    ipf_shutdown()

## 'Sticky'/'Seamless' Remote Excution 

via Input Transformers.  These can make cells set execute remotely by default.

**WARNINGS**: 
1. Solve it is not intended to work with people modifying input transformers So be wary. Nevertheless, this seems to work.
2. If they're commands that you definitely want to execute locally, maybe run `%unset_sticky` just to be sure first.


In [ ]:
#| export
#| notest
gip = get_ipython()

@register_line_magic
def set_sticky(_):
    "Makes code cells execute remotely, via input transformer"
    assert _ipf_kc is not None, "Need an active remote kernel connection" 
    for f in gip.input_transformers_cleanup[:]:   # gaurd against appending twice
        if getattr(f, '__name__', '') == '_execute_remotely':
            print("Already executing remotely") 
            return 
    gip.input_transformers_cleanup.append(_execute_remotely)
    print('Code cells will now execute remotely.')

In [ ]:
#| export
#| notest
@register_line_magic
def unset_sticky(_):
    "Un-sticks remote execution for code cells" 
    for f in gip.input_transformers_cleanup[:]:  
        if getattr(f, '__name__', '') == '_execute_remotely':
            gip.input_transformers_cleanup.remove(f)
    print("Code cells will now run locally.") 



In [ ]:
%unset_remote 
print()
%set_remote {port}
%set_sticky

Code cells will now run locally.
Shutting down remote kernel



/app/data/.ssh/config file updated.


Success: remote kernel started
Code cells will now execute remotely.


In [ ]:
# remote execution
import socket 
hostname = socket.gethostname()
print("Hello from",hostname) 

Hello from Chonk


In [ ]:
%unset_sticky

Code cells will now run locally.


In [ ]:
# local execution
import socket 
hostname = socket.gethostname()
print("Hello from",hostname) 

Hello from 909e93a3a71e


In [ ]:
#| hide

#import nbdev; nbdev.nbdev_export()
!nbdev_export --procs scrub_magics